<H1>The Transportation Problem</H1>
* Consider a set of warehouses each with a given inventory of widgets, and a set of demand centers each with a given demand for widgets. 
* How many widgets should we ship from each warehouse to each demand center such that all demand is satisfied and shipping costs are minimized?

![Transportation Problem](transportation_problem.png)
<H3> Sets and Indices </H3>
* $i \in I$: Warehouses
* $j \in J$: Customers (demand centers)

<H3>Data</H3>
* $u_i$: capacity for warehouse $i$ (widgets)
* $d_j$: demand at demand center $j$ (widgets)
* $c_{ij}$: shipping cost from warehouse $i$ to customer $j$ ($/widget)

<H3>Decision Variables</H3>
* $x_{ij}$: number of widgets to ship from warehouse $i$ to customer $j$

<H3>Linear Programming Formulation</H3>
\begin{eqnarray}
\min_{x} && \sum_{i \in I} \sum_{j \in J} c_{ij} x_{ij} \;\; \mbox{(minimize shipping costs)} \nonumber \\
\mbox{s.t.} && \sum_{i \in I} x_{ij} = d_j,\;\;j \in J \;\; \mbox{(satisfy demand)}\nonumber \\
&& \sum_{j \in J} x_{ij} \le u_i,\;\;i \in I \;\; \mbox{(don't exceed capacity)} \nonumber \\
&& x_{ij} \ge 0, \;\;i \in I,\;j \in J \;\; \mbox{(ship nonnegative quantities)} \nonumber
\end{eqnarray}

<H3>Inputs</H3>

We'll consider a test data set with four demand centers that have the following demands:

In [1]:
demands = [15, 18, 14, 20]

And with five warehouses that have the following widget capacities:

In [2]:
capacities = [20, 22, 17, 19, 18]

In [3]:
warehouses = range(len(capacities))
print (warehouses)

range(0, 5)


Finally, we'll need to know the per unit shipping costs between each warehouse and demand center.

In [4]:
ship_costs =  [[4000, 2500, 1200, 2200],
               [2000, 2600, 1800, 2600],
               [3000, 3400, 2600, 3100],
               [2500, 3000, 4100, 3700],
               [4500, 4000, 3000, 3200]]

<H3>Decision Variables</H3>

For each warehouse, customer pair, we'll need to decide how many units to ship. After adding those variables, we can think about how to generate the linear expressions needed to create the model.

In [5]:
def get_ship_vars(model, warehouses, customers):
    return {(warehouse, customer): model.addVar(...)
                                   for customer in customers
                                   for warehouse in warehouses}

<H3>Linear Expressions</H3>

This transportation model has two families of constraints. It is important to understand for each constraint which index set is being looped over, and which is being summed over. For demands, there exists one constraint per customer and each constraint involves a sum over the warehouses, and for supplies there exists one constraint per warehouse and each constraint involves a sum over the customers.

Let's create a test model and experiment with generating expressions out of the ship variables.

In [6]:
import gurobipy as grb
m = grb.Model()
warehouses = range(len(capacities))
customers = range(len(demands))
to_ship = get_ship_vars(m, warehouses, customers)
m.update()
to_ship

Using license file C:\Users\liaml\gurobi.lic
Academic license - for non-commercial use only


{(0, 0): <gurobi.Var ship.w0.c0>,
 (1, 0): <gurobi.Var ship.w1.c0>,
 (2, 0): <gurobi.Var ship.w2.c0>,
 (3, 0): <gurobi.Var ship.w3.c0>,
 (4, 0): <gurobi.Var ship.w4.c0>,
 (0, 1): <gurobi.Var ship.w0.c1>,
 (1, 1): <gurobi.Var ship.w1.c1>,
 (2, 1): <gurobi.Var ship.w2.c1>,
 (3, 1): <gurobi.Var ship.w3.c1>,
 (4, 1): <gurobi.Var ship.w4.c1>,
 (0, 2): <gurobi.Var ship.w0.c2>,
 (1, 2): <gurobi.Var ship.w1.c2>,
 (2, 2): <gurobi.Var ship.w2.c2>,
 (3, 2): <gurobi.Var ship.w3.c2>,
 (4, 2): <gurobi.Var ship.w4.c2>,
 (0, 3): <gurobi.Var ship.w0.c3>,
 (1, 3): <gurobi.Var ship.w1.c3>,
 (2, 3): <gurobi.Var ship.w2.c3>,
 (3, 3): <gurobi.Var ship.w3.c3>,
 (4, 3): <gurobi.Var ship.w4.c3>}

We'll pick a customer first, say customer 0, and generate an expression for the number of units that customer received.

In [7]:
grb.quicksum(...)

<gurobi.LinExpr: ship.w0.c0 + ship.w1.c0 + ship.w2.c0 + ship.w3.c0 + ship.w4.c0>

This fixes customer $c_0$, and sums over the warehouses $w_0,\ldots,w_4$. If this works for customer $c_0$, it should work for the rest of the customers if we add an outer loop.

In [8]:
for customer in customers:
    print (grb.quicksum(...))

<gurobi.LinExpr: ship.w0.c0 + ship.w1.c0 + ship.w2.c0 + ship.w3.c0 + ship.w4.c0>
<gurobi.LinExpr: ship.w0.c1 + ship.w1.c1 + ship.w2.c1 + ship.w3.c1 + ship.w4.c1>
<gurobi.LinExpr: ship.w0.c2 + ship.w1.c2 + ship.w2.c2 + ship.w3.c2 + ship.w4.c2>
<gurobi.LinExpr: ship.w0.c3 + ship.w1.c3 + ship.w2.c3 + ship.w3.c3 + ship.w4.c3>


This should have generated a linear expression for each customer $c_0,\ldots,c_3$. If so, these are the LinExpr objects we need to construct demand constraints. The takeaway is that the set we are summing over should participate in the inner loop, and the set we are writing the constraint over should participate in the outer loop.

To generate expressions for the supply constraints, we'll need to fix a warehouse and sum over the customers, so we'll want customers on the inner loop and warehouses on the outer.

In [9]:
for warehouse in warehouses:
    print (grb.quicksum(...))

<gurobi.LinExpr: ship.w0.c0 + ship.w0.c1 + ship.w0.c2 + ship.w0.c3>
<gurobi.LinExpr: ship.w1.c0 + ship.w1.c1 + ship.w1.c2 + ship.w1.c3>
<gurobi.LinExpr: ship.w2.c0 + ship.w2.c1 + ship.w2.c2 + ship.w2.c3>
<gurobi.LinExpr: ship.w3.c0 + ship.w3.c1 + ship.w3.c2 + ship.w3.c3>
<gurobi.LinExpr: ship.w4.c0 + ship.w4.c1 + ship.w4.c2 + ship.w4.c3>


Each LinExpr object we just generated should consider a single warehouse and sum over all customers.

<H3>Constraints</H3>

Generating the correct LinExpr objects for each demand and supply is most of the challenge. We can turn each LinExpr into a constraint by using the overloaded $<=$, $>=$, and $==$ operators.

For the demand constraints, we'll loop over the customers and sum over the warehouses.

Note: We are using Python's list comprehension syntax here, which puts the outer for loop inside the []. It is still the case that the outer loop iterates over the customer, and the inner loop sums over the warehouses.

In [10]:
def get_demand_constrs(model, to_ship, demands):
    #Return a list of demand constraints for each cutomer
    return [model.addConstr(...)
           for customer, demand in enumerate(demands)]

We can write a similar method to generate the capacity constraints.

In [11]:
def get_capacity_constrs(model, to_ship, capacities):
    #Return a list of capacity constraints for each warehouse
    return [model.addConstr(...)
            for warehouse, capacity in enumerate(capacities)]

<H3>Solving the Model</H3>

Now we have the building blocks needed to build and solve a transportation model. Let's put it all together.

In [12]:
import gurobipy as grb
GRB = grb.GRB
def solve_transportation_model(capacities, demands):
    model = grb.Model()
    warehouses = range(len(capacities))
    customers = range(len(demands))
    to_ship = get_ship_vars(model, warehouses, customers)
    model.update()
    capacity_constrs = get_capacity_constrs(model, to_ship, capacities)
    demand_constrs = get_demand_constrs(model, to_ship, demands)
    
    #Minimize the total cost to ship, using the shipping costs and the amount shipped on each arc
    model.setObjective(grb.quicksum(...))
    
    model.optimize()
    if model.Status == GRB.OPTIMAL:
        for (warehouse, customer), var in sorted(to_ship.items()):
            if var.X > 1e-4:
                print ("Ship", var.X, "units from warehouse", warehouse, "to customer", customer)

In [13]:
solve_transportation_model(capacities, demands)

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 9 rows, 20 columns and 40 nonzeros
Model fingerprint: 0xd56d0caf
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+03, 5e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 2e+01]
Presolve time: 0.01s
Presolved: 9 rows, 20 columns, 40 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.3580000e+05   3.200000e+01   0.000000e+00      0s
       5    1.5390000e+05   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.01 seconds
Optimal objective  1.539000000e+05
Ship 14.0 units from warehouse 0 to customer 2
Ship 6.0 units from warehouse 0 to customer 3
Ship 14.0 units from warehouse 1 to customer 0
Ship 8.0 units from warehouse 1 to customer 3
Ship 6.0 units from warehouse 2 to customer 3
Ship 1.0 units from warehouse 3 to customer 0
Ship 18.0 units from warehouse 3 to customer 1
